### Инициализация

In [ ]:
#что бы подключить гугл диск к проекту
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive

/content/drive/My Drive


# класс зданиe

In [ ]:
class Building():
  def __init__(self, coordinates, distance, azimuth, population, amenity = None):
        self.coordinates = coordinates
        self.distance = distance
        self.azimuth = azimuth
        self.population = population
        self.amenity = amenity



# Поиск средней координаты


In [ ]:
def getMediumCoorinates(input: []):
  firstC = 0
  secondC = 0
  for coord in input:
    firstC += coord[0]
    secondC += coord[1]
  return (firstC/len(input), secondC/len(input))


## **Поиск географического центра владивостока:**


In [ ]:
vladivostok = open('/content/drive/My Drive/data/Vladivostok_buildings_POI_1/Vladivostok_adm.geojson', 'r')
vlad = json.loads(vladivostok.read())
centreOfVlad = getMediumCoorinates(vlad["features"][0]["geometry"]["coordinates"][0])
print(centreOfVlad)

(131.9971945870371, 43.15664895185186)


In [ ]:
!pip install haversine
from haversine import haversine

In [ ]:
import math
def azimuth(x1,y1,x2,y2):
  a = 90 - math.atan2(y2-y1, x2-x1)*180/math.pi
  if (a<0): 
    a += 360
  return a


68.8112037557741

# Перевод всех зданий в полярную СК


In [273]:
buildings= open('/content/drive/My Drive/data/Vladivostok_buildings_POI_1/buildings.geojson', 'r')
building = json.loads(buildings.read())
houses = []
allDistances = []

for i in building["features"]:
  if i["properties"]["population"] != None:
    allDistances.append( haversine(centreOfVlad, i["geometry"]["coordinates"][0][0][0]))
    houses.append(Building(i["geometry"]["coordinates"][0][0][0] ,
                           haversine(centreOfVlad, i["geometry"]["coordinates"][0][0][0]), 
                           azimuth(centreOfVlad[0], centreOfVlad[1], i["geometry"]["coordinates"][0][0][0][0], i["geometry"]["coordinates"][0][0][0][1]), 
                           i["properties"]["population"],
                           str(i["properties"]["amenity_place"] )))

# POIs = open('/content/drive/My Drive/data/Vladivostok_buildings_POI_1/POI.geojson', 'r')
# POI = json.loads(POIs.read())
# for i in POI["features"]:
#   houses.append(Building(i["geometry"]["coordinates"] ,
#                            haversine(centreOfVlad, i["geometry"]["coordinates"]), 
#                            azimuth(centreOfVlad[0], centreOfVlad[1], i["geometry"]["coordinates"][0], i["geometry"]["coordinates"][1]), 
#                            1,
#                            str(i["properties"]["amenity"] )))


AllSortedDistances = sorted(allDistances)
buildingsByDistance = sorted(houses, key=lambda house: house.distance)



## Поиск точек внутри круга:

In [275]:
import numpy as np

def find_angle(distance,r=0.5):
    angle = math.asin(r/distance)
    return angle * 180/math.pi

def IndexesInRadius(buildingParameter, sortedDistances: [], radius = 0.5): 
  maxIndex = np.argmin(np.abs(np.array(sortedDistances)- buildingParameter - radius))
  minIndex = np.argmin(np.abs(np.array(sortedDistances)- buildingParameter + radius))
  return (minIndex, maxIndex+1)

def GetIndexesInKm(building: Building):
  sortedByAzimuth = []
  allInRing = IndexesInRadius(building.distance, AllSortedDistances)
  sortedInRing = sorted(buildingsByDistance[allInRing[0]:allInRing[1]] , key=lambda house: house.azimuth)
  for i in sortedInRing:
    sortedByAzimuth.append(i.azimuth)

  borderIndexes = IndexesInRadius(building.azimuth, sortedByAzimuth, find_angle(building.distance))
  return borderIndexes , sortedInRing

def GetBuildingsAroundPoint(building: Building):
  indexes = GetIndexesInKm(building)
  borders = indexes[0]
  array = indexes[1]
  return array[borders[0]:borders[1]]


# for i in GetBuildingsAroundPoint(buildingsByDistance[1111]):   #TEST
#   print(i.coordinates, i.distance, i.azimuth)

# Подсчет весов для каждой точки


In [276]:
list_of_places_with_weight={"None":0.7, "bus_station":0, "customs":0, "atm":0.9, "cinema":0, "ferry_terminal":0,
                            "nightclub":0, "library":0, "parking":0, "courthouse":0, "fire_station":0, "cafe":0.25,
                            "bank":0.75, "dentist":0, "driving_school":0, "pharmacy":0.1, "post_office":0, "fuel":0, 
                            "doctors":0, "restaurant":0.2, "veterinary":0, "car_wash":0, "hospital":0.4, "school":0, 
                            "place_of_worship":0, "fast_food":0.5, "clinic":0, "Issuing orders":0, "pub":0, 
                            "waste_disposal":0, "toilets":0, "bar":0, "police":0, "telephone":0, "waste_basket":0, 
                            "beauty":0, "ski_rental":0, "equipment_rental":0, "theatre":0, "kindergarten":0, 
                            "arts_centre":0.15, "community_centre":0.6, "vending_machine":0.7, "social_facility":0,
                            "bench":0, "shelter":0, "public_building":0, "bbq":0, "post_box":0, "drinking_water":0,
                            "grit_bin":0, "vehicle_ramp":0, "university":0.55, "food_court":0.58,
                            "payment_terminal":0.95, "studio":0, "marketplace":0.75, "fountain":0, "recycling":0, 
                            "public_bookcase":0, "townhall":0, "car_rental":0, "parking_entrance":0, 
                            "vehicle_inspection":0, "childcare":0, "boat_rental":0, "clock":0, "shower":0,
                            "embassy":0, "college":0.45, "charging_station":0, "hunting_stand":0, 'rescue_station': 0, 
                            "bureau_de_change":0, "boat_storage":0, "yacht_club":0, "music_school":0, "stripclub":0, "prison": 0}

buildingWithWeight= {}

for mainBuilding in buildingsByDistance:
  sum = 0
  for buildingAround in GetBuildingsAroundPoint(mainBuilding):
    if (mainBuilding in buildingWithWeight): 
      buildingWithWeight[mainBuilding] += (buildingAround.population % 50)*list_of_places_with_weight[mainBuilding.amenity]
    else:
      buildingWithWeight[mainBuilding] = buildingAround.population*list_of_places_with_weight[mainBuilding.amenity]
      
# n=0
# for i in buildingWithWeight:
#   if n>=100: break
#   print( buildingWithWeight[i], i.coordinates  )
#   n+=1

a = {k: v for k, v in sorted(buildingWithWeight.items(), key=lambda item: item[1] , reverse=True)}
print(len(a))
n=0
for i in a:
  if n>=1000: break
  print( i.coordinates , end = ', '  )
  n+=1

3608
[131.9114142, 43.1518917], [131.9101206, 43.1517096], [131.9124188, 43.1513338], [131.9101878, 43.1501766], [131.9110318, 43.1502935], [131.9125505, 43.1504633], [131.9122404, 43.1506908], [131.9149743, 43.1494586], [131.914974, 43.1494581], [131.9115796, 43.1504673], [131.9144641, 43.1519033], [131.913394, 43.149771], [131.9098854, 43.1498554], [131.911352, 43.1499419], [131.9130484, 43.1499839], [131.9107839, 43.1508055], [131.9130065, 43.1526974], [131.9125465, 43.1501422], [131.913867, 43.1517367], [131.9111947, 43.1514181], [131.9126949, 43.1529078], [131.9121637, 43.1502512], [131.9119462, 43.1500263], [131.912369, 43.1523489], [131.9131808, 43.1504296], [131.9131158, 43.150106], [131.912326, 43.1517463], [131.9101467, 43.148069], [131.9105775, 43.1519356], [131.9108503, 43.1480951], [131.9117639, 43.1496277], [131.9135112, 43.1509333], [131.912779, 43.1530954], [131.9103424, 43.1512143], [131.9115173, 43.1517387], [131.9096497, 43.1497463], [131.9129632, 43.1496052], [131.9